In [7]:
import pygame
import random
import sys
import time
import psycopg2

WIDTH, HEIGHT = 640, 640
PIXELS = 32
SQUARES_X, SQUARES_Y = WIDTH // PIXELS, HEIGHT // PIXELS

EXCLUDED_COLORS = [(0, 255, 255), (255, 215, 0), (0, 0, 255), (255, 0, 0)]
POSSIBLE_BG_COLORS = [(0, 255, 0), (255, 165, 0), (128, 0, 128), (255, 192, 203), (144, 238, 144), (211, 211, 211)]
BG_COLORS = [color for color in POSSIBLE_BG_COLORS if color not in EXCLUDED_COLORS]
DEFAULT_BG_COLOR = (156, 210, 54)

RED = (255, 0, 0)
CYAN = (0, 255, 255)
GOLD = (255, 215, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)

conn = psycopg2.connect(
    host="ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech",
    dbname="neondb",
    user="neondb_owner",
    password="npg_Um02TyoiHXhw"
)

def get_user(username):
    cur = conn.cursor()
    cur.execute("SELECT id FROM \"user\" WHERE username = %s", (username,))
    row = cur.fetchone()
    if row:
        user_id = row[0]
        cur.execute("SELECT level, record FROM user_score WHERE user_id = %s", (user_id,))
        stats = cur.fetchone()
        return user_id, stats[0], stats[1]
    else:
        cur.execute("INSERT INTO \"user\" (username) VALUES (%s) RETURNING id", (username,))
        user_id = cur.fetchone()[0]
        cur.execute("INSERT INTO user_score (user_id, level, record) VALUES (%s, 1, 0)", (user_id,))
        conn.commit()
        return user_id, 1, 0

def update_score(user_id, level, score):
    cur = conn.cursor()
    cur.execute("SELECT record FROM user_score WHERE user_id = %s", (user_id,))
    current = cur.fetchone()[0]
    if score > current:
        cur.execute("UPDATE user_score SET level = %s, record = %s WHERE user_id = %s", (level, score, user_id))
        conn.commit()

class Snake:
    def __init__(self, level=1, record=0):
        self.head = [SQUARES_X // 2, SQUARES_Y // 2]
        self.body = [tuple(self.head)]
        self.dir = (1, 0)
        self.grow = False
        self.points = 0
        self.level = level
        self.record = record
        self.speed = 5 + (self.level - 1) * 0.5
        self.bg_color = DEFAULT_BG_COLOR if self.level < 3 else random.choice(BG_COLORS)
        self.font = pygame.font.SysFont("Arial", 24, bold=True)

    def update_dir(self, key):
        if key == pygame.K_UP and self.dir != (0, 1): self.dir = (0, -1)
        elif key == pygame.K_DOWN and self.dir != (0, -1): self.dir = (0, 1)
        elif key == pygame.K_LEFT and self.dir != (1, 0): self.dir = (-1, 0)
        elif key == pygame.K_RIGHT and self.dir != (-1, 0): self.dir = (1, 0)

    def move(self):
        new = (self.body[0][0] + self.dir[0], self.body[0][1] + self.dir[1])
        if new in self.body or not (0 <= new[0] < SQUARES_X and 0 <= new[1] < SQUARES_Y):
            return False
        self.body.insert(0, new)
        if not self.grow:
            self.body.pop()
        else:
            self.grow = False
        return True

    def eat(self, points):
        self.points += points
        self.grow = True
        if self.points >= self.level * 5:
            self.level += 1
            self.speed += 0.5
            if self.level >= 3 and (self.level - 3) % 3 == 0:
                self.bg_color = random.choice(BG_COLORS)

    def draw(self, screen):
        for x, y in self.body:
            pygame.draw.rect(screen, BLUE, (x * PIXELS, y * PIXELS, PIXELS, PIXELS))

    def draw_info(self, screen, time_left):
        info = self.font.render(f"Score: {self.points} Level: {self.level} Record: {self.record}", True, BLACK)
        timer_lbl = self.font.render(f"Time: {int(time_left)}s", True, BLACK)
        screen.blit(info, (10, 10))
        screen.blit(timer_lbl, (WIDTH - 130, 10))

    def reset(self):
        self.__init__()

class Apple:
    def __init__(self):
        self.spawn()

    def spawn(self):
        chance = random.randint(1, 100)
        if chance <= 50:
            self.color, self.points = RED, 1
        elif chance <= 80:
            self.color, self.points = CYAN, 3
        else:
            self.color, self.points = GOLD, 5
        self.pos = (random.randint(0, SQUARES_X - 1), random.randint(0, SQUARES_Y - 1))
        self.spawn_time = time.time()

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, (self.pos[0] * PIXELS, self.pos[1] * PIXELS, PIXELS, PIXELS))

    def time_left(self):
        return max(0, 10 - (time.time() - self.spawn_time))

def draw_bg(screen, color):
    screen.fill(color)
    for y in range(SQUARES_Y):
        for x in range(SQUARES_X):
            if (x + y) % 2 == 0:
                rect = pygame.Rect(x * PIXELS, y * PIXELS, PIXELS, PIXELS)
                pygame.draw.rect(screen, tuple(max(0, c - 10) for c in color), rect)

def draw_home_button(screen, home_img):
    home_rect = home_img.get_rect(topright=(WIDTH - 10, 10))
    screen.blit(home_img, home_rect)
    return home_rect

def draw_menu(screen):
    pygame.draw.rect(screen, (200, 200, 200), (WIDTH // 2 - 100, HEIGHT // 2 - 50, 200, 120), border_radius=10)
    font = pygame.font.SysFont("Arial", 24, bold=True)
    save_btn = pygame.draw.rect(screen, (100, 255, 100), (WIDTH // 2 - 80, HEIGHT // 2 - 30, 160, 40), border_radius=8)
    exit_btn = pygame.draw.rect(screen, (255, 100, 100), (WIDTH // 2 - 80, HEIGHT // 2 + 30, 160, 40), border_radius=8)
    screen.blit(font.render("Save", True, BLACK), (WIDTH // 2 - 20, HEIGHT // 2 - 25))
    screen.blit(font.render("Exit", True, BLACK), (WIDTH // 2 - 20, HEIGHT // 2 + 35))
    return save_btn, exit_btn

def draw_username_input(screen, username):
    font = pygame.font.SysFont("Arial", 32, bold=True)
    input_box = pygame.Rect(WIDTH // 2 - 150, HEIGHT // 2 - 50, 300, 50)
    pygame.draw.rect(screen, (255, 255, 255), input_box, border_radius=5)
    text_surface = font.render(username, True, (0, 0, 0))
    screen.blit(text_surface, (input_box.x + 5, input_box.y + 5))
    return input_box

def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Snake Game")

    username = ""
    input_active = True
    user_id, level, record = None, 1, 0

    snake = None
    apple = None
    clock = pygame.time.Clock()

    home_img = pygame.image.load("home.png")
    home_img = pygame.transform.scale(home_img, (32, 32))
    menu_open = False
    paused = False 

    running = True
    while running:
        clock.tick(10)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                if user_id is not None:
                    update_score(user_id, snake.level, snake.points)
                running = False
            elif event.type == pygame.KEYDOWN:
                if input_active:
                    if event.key == pygame.K_BACKSPACE:
                        username = username[:-1]
                    elif event.key == pygame.K_RETURN and username != "":
                        input_active = False
                        user_id, level, record = get_user(username)
                        snake = Snake(level, record)
                        apple = Apple()
                    else:
                        username += event.unicode
                else:
                    if event.key == pygame.K_x:
                        menu_open = not menu_open
                        paused = not paused
                    elif not paused:
                        snake.update_dir(event.key)

        if not input_active and not paused:
            if not snake.move():
                update_score(user_id, snake.level, snake.points)
                snake = Snake()
                apple.spawn()

            if (snake.body[0][0], snake.body[0][1]) == apple.pos:
                snake.eat(apple.points)
                apple.spawn()

            if apple.time_left() <= 0:
                apple.spawn()

        screen.fill(BLACK)
        if input_active:
            draw_username_input(screen, username)
            font = pygame.font.SysFont("Arial", 32, bold=True)
            prompt_text = font.render("Enter your username:", True, (255, 255, 255))
            screen.blit(prompt_text, (WIDTH // 2 - prompt_text.get_width() // 2, HEIGHT // 2 - 100))
        else:
            draw_bg(screen, snake.bg_color)
            apple.draw(screen)
            snake.draw(screen)
            snake.draw_info(screen, apple.time_left())

            draw_home_button(screen, home_img)
            if menu_open:
                save_btn, exit_btn = draw_menu(screen)
                mouse_pos = pygame.mouse.get_pos()
                if save_btn.collidepoint(mouse_pos):
                    update_score(user_id, snake.level, snake.points)
                elif exit_btn.collidepoint(mouse_pos):
                    update_score(user_id, snake.level, snake.points)
                    running = False

        pygame.display.update()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


SystemExit: 